<a href="https://colab.research.google.com/github/stillrahim/jupyter-exploration/blob/main/A08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install tensorflow tensorflow-hub tensorflow-datasets

# Import libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Check if GPU or TPU is available
if tf.test.gpu_device_name():
    print('✅ GPU Available:', tf.test.gpu_device_name())
else:
    print('⚠️ No GPU found. Enable GPU under Runtime > Change runtime type > GPU.')

if tf.config.list_physical_devices('TPU'):
    print("✅ TPU is available")
else:
    print("⚠️ TPU is not available")


In [ ]:
# Load Pascal VOC 2007 dataset
dataset, info = tfds.load('voc/2007', with_info=True, as_supervised=True)
train_ds, test_ds = dataset['train'], dataset['test']

print("Classes:", info.features['labels'].names)
print("Number of training examples:", info.splits['train'].num_examples)
print("Number of test examples:", info.splits['test'].num_examples)


In [ ]:
import matplotlib.pyplot as plt

def show_sample(dataset):
    for image, label in dataset.take(1):
        plt.figure(figsize=(5, 5))
        plt.imshow(image)
        plt.title("Sample Image from Pascal VOC 2007")
        plt.axis('off')
        plt.show()

show_sample(train_ds)


In [ ]:
# Load SSD MobileNetV2 from TensorFlow Hub (transfer learning)
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
print("✅ Model Loaded Successfully!")


In [ ]:
def preprocess_image(image):
    img = tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]
    return img

# Example: take one image from test set
for sample in test_ds.take(1):
    image, label = sample
    input_img = preprocess_image(image)


In [ ]:
import matplotlib.patches as patches

def visualize_detection(image, boxes, classes, scores, threshold=0.5):
    fig, ax = plt.subplots(1, figsize=(10, 8))
    ax.imshow(image)

    for box, cls, score in zip(boxes[0], classes[0], scores[0]):
        if score >= threshold:
            ymin, xmin, ymax, xmax = box
            rect = patches.Rectangle(
                (xmin * image.shape[1], ymin * image.shape[0]),
                (xmax - xmin) * image.shape[1],
                (ymax - ymin) * image.shape[0],
                linewidth=2, edgecolor='lime', facecolor='none'
            )
            ax.add_patch(rect)
            ax.text(
                xmin * image.shape[1], ymin * image.shape[0] - 10,
                f'Class {int(cls)}: {score:.2f}', color='lime', fontsize=10
            )

    plt.title("Object Detection Results")
    plt.axis('off')
    plt.show()

visualize_detection(image, detection_boxes, detection_classes, detection_scores)


In [ ]:
model.save('/content/drive/MyDrive/object_detection_model')
print("✅ Model saved successfully to Google Drive!")
